For the final model, I decided to pick logistic regression because it looked very promising and seemed to give good results.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import combinations
from itertools import permutations

import random

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt


In [3]:
word_data = pd.read_csv('../data/unigram_freq.csv')
word_data_top1000 = word_data.head(1000)

wheel_data = pd.read_csv('../data/wheeldata.csv')
wheel_data_nodup = wheel_data.drop_duplicates(subset=['Puzzle'])


In [7]:
# regression set up - duplicated from other file

wheel_data_nodup['Puzzle'] = wheel_data_nodup.Puzzle.astype('str')

excluded = {'R', 'S', 'T', 'L', 'N', 'E', ' '}
letters = [chr(i) for i in range(65, 91) if chr(i) not in excluded]

vowels = {'A', 'E', 'I', 'O', 'U'}
consonants = set(letters) - vowels


def get_feature_vector(puzzle, guess_set):
    puzzle = puzzle.upper()
    letter_counts = [puzzle.count(l) for l in letters]
    guess_vector = [1 if l in guess_set else 0 for l in letters]
    return letter_counts + guess_vector

def get_reveal_score(puzzle, guess_set): # we make a guess with a combo of letters 
    return sum(1 for c in puzzle.upper() if c in guess_set and c not in excluded)/len(puzzle)

X_combinations = []
y_regress = []

guess_list = [] # this list stores all the guesses we used

for idx, row in wheel_data_nodup.iterrows():
    puzzle = row['Puzzle']
    puzzle_letters = set(puzzle.upper()) - excluded

    guesses = []
    for i in range(1000):
        cons = random.sample(list(consonants), 3)
        vwls = random.sample(list(vowels), 1)
        guesses.append(cons+vwls)

    for guess in guesses:
        guess_set = set(guess)
        features = get_feature_vector(puzzle, guess_set)
        score = get_reveal_score(puzzle, guess_set)
        
        X_combinations.append(features)
        y_regress.append(score)
        guess_list.append(''.join(guess))

C:\Users\axylo\AppData\Local\Temp\ipykernel_19480\841453476.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wheel_data_nodup['Puzzle'] = wheel_data_nodup.Puzzle.astype('str')


In [14]:
# logistic regression model code

# classification target
y_class = [1 if score >= 0.5 else 0 for score in y_regress] 

X_train, X_test, y_train, y_test = train_test_split(X_combinations, y_class, test_size=0.2, random_state=42)

# fit to model
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

# predict probabilities and classes
y_pred_proba = log_model.predict_proba(X_test)[:, 1]
y_pred = log_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Accuracy Score: ", acc)

Accuracy Score:  0.9990677374301676
